<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5955389ba3a23389cda71ead491007faa6c6ee3fcbbde74266fe366bca1057ff
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-13 11:18:22
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.40 C
-------------------
Today PnL: -1.46 L (-1.03%)
Current PnL: -21.22 L (-14.14%)
CY Booked + Current PnL: -9.78 L (-6.52%)
-------------------
Total profit:  1.52 L
Total loss:  -22.75 L
-------------------
Total Booked + Current PnL: 17.20 L (13.97%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.15%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.73 L (63.22%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.83%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.52,118593.0,18089.0,11017.0,-0.24,18.00,9.29,28.96,79.0,1.64,0.85,58.08,MH,ATH,METALS
19,CIPLA,1662.74,-20.12,56.0,X-LC,3.74,212747.0,8247.0,15041.0,-0.57,4.03,7.07,11.39,10.0,0.55,1.52,13.35,X40N,ATH,PHARMA
77,TTKPRESTIG,770.00,94.45,43.0,M-SC,1.24,83748.0,-17029.0,17118.0,0.64,-16.90,20.44,0.09,245.0,-0.99,0.60,10.03,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.94,49.0,X-MC,3.31,189800.0,25130.0,18202.0,-2.62,15.26,9.59,26.31,89.0,1.38,1.36,24.46,X40N,NTT,AC
84,VOLTAS,1530.00,-1.92,49.0,X-MC,1.02,205740.0,13998.0,23763.0,-3.22,7.30,11.55,19.69,99.0,0.59,1.47,14.59,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIS,528.00,1991.71,37.0,H-SC,3.83,84405.0,-26627.0,50238.0,1.77,-23.98,59.52,21.26,156.0,-0.53,0.60,14.04,OX40N,NTT,MISC
37,INDIAMART,4810.62,-54.27,45.0,H-SC,8.52,127815.0,4479.0,127150.0,0.84,3.63,99.48,106.72,119.0,0.04,0.91,27.86,AR,ATH,MISC
76,TRIDENT,48.00,-9.93,44.0,M-SC,1.62,70024.0,-22230.0,48534.0,0.78,-24.10,69.31,28.51,224.0,-0.46,0.50,19.82,XR,NTT,TEXTILES
77,TTKPRESTIG,770.00,94.45,43.0,M-SC,1.24,83748.0,-17029.0,17118.0,0.64,-16.90,20.44,0.09,245.0,-0.99,0.60,10.03,OX40N,NTT,DURABLES
35,ICICIPRULI,790.00,-21.22,49.0,X-MC,2.02,135476.0,-312.0,43068.0,0.29,-0.23,31.79,31.48,107.0,-0.01,0.97,11.86,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-5.39,37.0,H-LC,1.25,151935.0,-34160.0,81635.0,-3.45,-18.36,53.73,25.51,15.0,-0.42,1.09,11.96,AR,ATH,ELECTRICAL
84,VOLTAS,1530.0,-1.92,49.0,X-MC,1.02,205740.0,13998.0,23763.0,-3.22,7.30,11.55,19.69,99.0,0.59,1.47,14.59,XY25,NTT,AC
26,GREENPANEL,537.0,227.52,54.0,M-SC,4.48,146327.0,-36751.0,115730.0,-3.21,-20.07,79.09,43.14,230.0,-0.32,1.05,37.64,XY24,NTT,MISC
55,RAJOOENG,143.1,-40.86,46.0,H-SC,16.77,89740.0,-12760.0,53359.0,-3.14,-12.45,59.46,39.61,114.0,-0.24,0.64,9.60,AR,ATH,MISC
47,LAOPALA,464.0,101.32,36.0,H-SC,1.19,92041.0,-39064.0,90311.0,-2.70,-29.80,98.12,39.09,142.0,-0.43,0.66,18.54,AR,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.0,43.54,48.0,M-MC,12.60,225995.0,1033.0,166626.0,-0.7,0.46,73.73,74.53,192.0,0.01,1.62,32.86,XY24,BTT,STEEL
86,WIPRO,420.0,-13.79,48.0,M-LC,5.84,151733.0,788.0,108671.0,-1.6,0.52,71.62,72.51,53.0,0.01,1.09,6.58,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,63.0,H-SC,11.31,131301.0,-11178.0,30698.0,-0.31,-7.85,23.38,13.70,163.0,-0.36,0.94,54.97,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,94.45,43.0,M-SC,1.24,83748.0,-17029.0,17118.0,0.64,-16.90,20.44,0.09,245.0,-0.99,0.60,10.03,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,47.0,H-SC,1.97,220527.0,-49140.0,85476.0,-1.00,-18.22,38.76,13.47,138.0,-0.57,1.58,12.13,XY24,NTT,PAINTS
18,CERA,9475.0,-25.54,30.0,H-SC,1.94,135964.0,-39939.0,81959.0,-0.93,-22.71,60.28,23.89,149.0,-0.49,0.97,17.42,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-25.22,42.0,H-MC,6.84,101175.0,-30660.0,72886.0,-1.53,-23.26,72.04,32.03,98.0,-0.42,0.72,13.32,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,48.0,M-MC,12.60,225995.0,1033.0,166626.0,-0.70,0.46,73.73,74.53,192.0,0.01,1.62,32.86,XY24,BTT,STEEL
32,HINDZINC,730.22,32.99,64.0,M-LC,8.23,216739.0,11663.0,100914.0,-0.01,5.69,46.56,54.89,52.0,0.12,1.55,31.15,X5K,ATH,METALS
79,UNIONBANK,163.00,-13.18,56.0,M-LC,9.34,157691.0,16851.0,27643.0,-0.42,11.96,17.53,31.59,66.0,0.61,1.13,41.62,XY24,NTT,BANKS
86,WIPRO,420.00,-13.79,48.0,M-LC,5.84,151733.0,788.0,108671.0,-1.60,0.52,71.62,72.51,53.0,0.01,1.09,6.58,XR,NTT,IT
11,BANKINDIA,190.00,-24.22,66.0,H-MC,12.33,194342.0,14534.0,97871.0,-0.32,8.08,50.36,62.52,88.0,0.15,1.39,44.92,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.52,118593.0,18089.0,11017.0,-0.24,18.00,9.29,28.96,79.0,1.64,0.85,58.08,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,66.0,H-MC,12.33,194342.0,14534.0,97871.0,-0.32,8.08,50.36,62.52,88.0,0.15,1.39,44.92,XR,NTT,BANKS
32,HINDZINC,730.22,32.99,64.0,M-LC,8.23,216739.0,11663.0,100914.0,-0.01,5.69,46.56,54.89,52.0,0.12,1.55,31.15,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.27,45.0,H-SC,8.52,127815.0,4479.0,127150.0,0.84,3.63,99.48,106.72,119.0,0.04,0.91,27.86,AR,ATH,MISC
86,WIPRO,420.00,-13.79,48.0,M-LC,5.84,151733.0,788.0,108671.0,-1.60,0.52,71.62,72.51,53.0,0.01,1.09,6.58,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-7.60,32.0,X-MC,0.91,229164.0,-18567.0,93636.0,-1.80,-7.49,40.86,30.30,92.0,-0.20,1.64,4.80,X40,ATH,ELECTRICAL
23,DABUR,735.00,-13.20,33.0,X-MC,3.55,234958.0,-13110.0,121520.0,-1.07,-5.28,51.72,43.70,102.0,-0.11,1.68,8.40,XY24,BTT,FMCG
5,ASIANPAINT,3465.66,-18.16,35.0,X-LC,5.47,206501.0,-45267.0,98480.0,0.27,-17.98,47.69,21.13,27.0,-0.46,1.48,11.08,X40,ATH,PAINTS
9,BAJAJHFL,181.50,-15.11,35.0,X-MC,5.14,178727.0,-21779.0,118925.0,-0.93,-10.86,66.54,48.45,90.0,-0.18,1.28,3.06,X40N,ATH,FINANCE
53,QUESS,986.00,-46.56,35.0,X-SC,37.72,53129.0,-11877.0,162803.0,-1.08,-18.27,306.43,232.18,198.0,-0.07,0.38,1.93,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-7.60,32.0,X-MC,0.91,229164.0,-18567.0,93636.0,-1.80,-7.49,40.86,30.30,92.0,-0.20,1.64,4.80,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,-1.92,49.0,X-MC,1.02,205740.0,13998.0,23763.0,-3.22,7.30,11.55,19.69,99.0,0.59,1.47,14.59,XY25,NTT,AC
73,TATAMOTORS,1065.00,-23.48,39.0,X-LC,1.20,242048.0,-34475.0,144551.0,-1.79,-12.47,59.72,39.81,54.0,-0.24,1.73,16.00,XY24,NTT,AUTO
65,SIEMENS,4671.50,-5.39,37.0,H-LC,1.25,151935.0,-34160.0,81635.0,-3.45,-18.36,53.73,25.51,15.0,-0.42,1.09,11.96,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.58,39.0,X-MC,1.68,136508.0,375.0,30755.0,-0.97,0.28,22.53,22.86,91.0,0.01,0.98,12.92,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-21.85,37.0,X-LC,10.48,281677.0,-34165.0,146838.0,-0.48,-10.82,52.13,35.67,5.0,-0.23,2.02,1.54,X40N,ATH,FMCG
53,QUESS,986.00,-46.56,35.0,X-SC,37.72,53129.0,-11877.0,162803.0,-1.08,-18.27,306.43,232.18,198.0,-0.07,0.38,1.93,XY24,NTT,MISC
22,COLPAL,3767.14,-5.42,37.0,X-MC,7.47,220830.0,-42535.0,155884.0,-0.88,-16.15,70.59,43.04,84.0,-0.27,1.58,2.52,XY25,ATH,FMCG
42,ITC,452.00,-39.61,42.0,X-LC,2.56,195380.0,-4758.0,25653.0,-0.81,-2.38,13.13,10.44,4.0,-0.19,1.40,3.05,X40,NTT,FMCG
9,BAJAJHFL,181.50,-15.11,35.0,X-MC,5.14,178727.0,-21779.0,118925.0,-0.93,-10.86,66.54,48.45,90.0,-0.18,1.28,3.06,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.56,35.0,X-SC,37.72,53129.0,-11877.0,162803.0,-1.08,-18.27,306.43,232.18,198.0,-0.07,0.38,1.93,XY24,NTT,MISC
56,RELAXO,1176.00,-40.36,37.0,X-SC,2.87,81235.0,-63925.0,139854.0,-0.57,-44.04,172.16,52.31,136.0,-0.46,0.58,8.74,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.72,40.0,X-SC,4.40,96114.0,-32556.0,77852.0,-0.94,-25.30,81.00,35.20,219.0,-0.42,0.69,10.32,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-5.38,49.0,X-SC,1.21,107640.0,5636.0,38288.0,-0.43,5.53,35.57,43.06,122.0,0.15,0.77,23.59,X40N,ATH,MISC
33,HONAUT,58357.33,-22.88,46.0,X-SC,2.19,108750.0,-19188.0,66327.0,-0.62,-15.00,60.99,36.84,143.0,-0.29,0.78,11.33,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.01,48.0,X-LC,12.18,282714.0,-63242.0,131490.0,-0.68,-18.28,46.51,19.73,1.0,-0.48,2.02,4.13,X40,ATH,IT
40,INFY,2275.00,-18.52,51.0,X-LC,7.64,317157.0,4269.0,167427.0,-1.71,1.36,52.79,54.87,3.0,0.03,2.27,8.10,X40,BTT,IT
42,ITC,452.00,-39.61,42.0,X-LC,2.56,195380.0,-4758.0,25653.0,-0.81,-2.38,13.13,10.44,4.0,-0.19,1.40,3.05,X40,NTT,FMCG
83,VBL,671.64,-21.85,37.0,X-LC,10.48,281677.0,-34165.0,146838.0,-0.48,-10.82,52.13,35.67,5.0,-0.23,2.02,1.54,X40N,ATH,FMCG
1,ABB,7934.00,-41.03,42.0,H-LC,7.44,246264.0,-15355.0,134559.0,-1.07,-5.87,54.64,45.57,7.0,-0.11,1.76,4.79,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,29.09,82464.0,-31085.0,71133.0,-2.63,-27.38,86.26,35.27,268.0,-0.44,0.59,101.83,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,41.0,H-SC,10.02,90240.0,-7740.0,29346.0,-1.31,-7.90,32.52,22.05,152.0,-0.26,0.65,31.17,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.52,118593.0,18089.0,11017.0,-0.24,18.00,9.29,28.96,79.0,1.64,0.85,58.08,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,63.0,H-SC,11.31,131301.0,-11178.0,30698.0,-0.31,-7.85,23.38,13.70,163.0,-0.36,0.94,54.97,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,64.0,M-SC,16.35,144936.0,-4284.0,148458.0,-0.73,-2.87,102.43,96.62,208.0,-0.03,1.04,56.85,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,64.0,M-SC,16.35,144936.0,-4284.0,148458.0,-0.73,-2.87,102.43,96.62,208.0,-0.03,1.04,56.85,XY25,NTT,FINANCE
69,SURYODAY,240.00,64.47,62.0,H-SC,9.89,151072.0,-27999.0,93725.0,-1.92,-15.64,62.04,36.71,135.0,-0.30,1.08,49.89,XR,NTT,BANKS
32,HINDZINC,730.22,32.99,64.0,M-LC,8.23,216739.0,11663.0,100914.0,-0.01,5.69,46.56,54.89,52.0,0.12,1.55,31.15,X5K,ATH,METALS
64,SHALBY,327.00,1183.89,56.0,M-SC,22.36,174532.0,-6611.0,52080.0,-0.08,-3.65,29.84,25.10,235.0,-0.13,1.25,39.87,XY24,NTT,HEALTHCARE
68,STARHEALTH,761.00,16.36,65.0,H-SC,7.81,259162.0,-21961.0,154823.0,-0.49,-7.81,59.74,47.26,144.0,-0.14,1.85,38.85,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.16
1,25,43.52
2,50,75.19


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.99
LC    32.31
MC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.01
X40      14.64
X40N     11.27
XR       10.96
XY25     10.41
AR        8.10
OX40N     7.58
X200      1.75
MH        1.64
X5K       1.55
SR        1.12
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.47
X-LC    22.93
X-MC    17.19
M-SC    12.55
H-MC     4.61
H-LC     4.60
X-SC     4.53
M-LC     3.77
M-MC     1.62
L-SC     1.44
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.81,-6.42,41.49
IT,12.83,-17.30,80.83
FINANCE,10.48,-15.39,71.11
MISC,6.91,-18.74,84.54
BANKS,6.46,-10.50,68.63
PAINTS,5.52,-19.75,37.60
ELECTRICAL,5.27,-11.86,51.10
INSURANCE,3.80,-4.12,43.05
AC,3.52,1.63,14.84


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3240678.0,22
XR,1378291.0,14
AR,1194860.0,9
X40,796181.0,10
X40N,661653.0,9
OX40N,585538.0,10
XY25,482875.0,7
SR,266033.0,2
X5K,100914.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3439707.0,25
M-SC,1285927.0,15
X-MC,1134041.0,12
X-LC,1037692.0,13
X-SC,548353.0,6
H-MC,364974.0,4
H-LC,297439.0,3
L-SC,259047.0,3
M-LC,237228.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1200436.0      6
           AR         813671.0      5
           XR         811317.0      7
M-SC       XY24       691974.0      6
X-MC       XY24       593184.0      4
X-LC       X40        494246.0      5
X-SC       X40N       307698.0      4
H-SC       OX40N      274381.0      4
           SR         266033.0      2
X-MC       X40        224083.0      4
X-LC       X40N       216828.0      3
H-LC       AR         216194.0      2
X-LC       XY24       214812.0      2
H-MC       XY24       183200.0      1
X-MC       XY25       179647.0      2
L-SC       XR         169832.0      2
M-MC       XY24       166626.0      1
M-SC       AR         164995.0      2
X-SC       XY24       162803.0      1
M-SC       OX40N      149056.0      4
           XY25       148458.0      1
X-MC       X40N       137127.0      2
M-SC       XR         131444.0      2
X-LC       XY25       111806.0      3
M-LC       XR         108671.0      1
           X5K        100914.0      1
H-MC       XR          97871.0      1
L-SC       OX40N       89215.0      1
H-LC       X200        81245.0      1
X-SC       X40         77852.0      1
H-SC       MH          73869.0      1
H-MC       OX40N       72886.0      1
L-MC       XR          59156.0      1
L-LC       XY25        42964.0      1
M-LC       XY24        27643.0      1
H-MC       MH          11017.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
